Your tasks are as follows:

1. Connect to the **Foursquare API**
2. Connect to the **Yelp API**. This API offers similar services as Foursquare.
3. For **each of the bike stations** in Part 1 (CityBikes API), query both APIs to retrieve information for the following in that location:
      - Restaurants or bars
      - Various POIs (points of interest) of your choice
4. Create a DataFrame for the Yelp results and Foursquare results.
5. Compare the quality of the Yelp and Foursquare API. For your location, which API gives you the most complete information/better coverage? 
      * NOTE: Your definition of 'coverage' is up to you. It could be simple 'number of POIs in the area', but it could also be something more specific like 'number of reviews per POI', or 'number of different attributes of each POI'.
Complete the yelp_foursquare_EDA.ipynb notebook to demonstrate how you executed the tasks above.

In [1]:
import requests
import os
import pandas as pd
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

## Retrieve information of NewYork_BikeStations in Part 1

In [2]:
df_NewYork_BikeStations = pd.read_csv('../data/NewYork_BikeStations.csv')

print(df_NewYork_BikeStations.shape)
df_NewYork_BikeStations.head()

(37, 4)


,Name_Station,Latitude_Station,Longitude_Station,Number_Of_Bikes
0,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",40.755226,-73.965826,16
1,"Icon Pkg: W 29 St, 11th Ave & 10th Ave",40.752384,-74.003960,13
2,"Icon Pkg: E 27 St, 2nd Ave & 3rd Ave",40.740531,-73.979577,20
3,Icon Pkg: King Street & Greenwich St,40.728319,-74.008412,8
4,"Icon Pkg: W 97 St, Cntrl Park W & Col.",40.792596,-73.965278,14


# A - Foursquare

In [3]:
# Call environment variable for FourSquare API key
access_token = os.environ["FOURSQUARE_API_KEY"]

# Create dictionary for headers
headers = {"Accept": "application/json",
           "Authorization": access_token}

In [4]:
# # SAMPLE params and full url:
# params = {
#   	"query": "coffee",
#   	"ll": "47.606,-122.349358",
#   	"open_now": "true",
#   	"sort":"DISTANCE"
# }
# url = f'https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}&limit={limit}'

## Send a request to Foursquare for all the bike stations in your city of choice 

## 1) EXPERIMENT with 1 bike_station to see the structure

### 1.1.) Try the request with params containing: lat/long, radius, query, max_price, sort, limit
* New york bike station (lat/long)
* radius             : 1000m
* looking for (query): coffee or restaurant
* with max_price          : level 3  (1: most affordable to  4: most expensive)
* with sort               : RATING   (Specifies the order in which results are listed)
* limit                   : 20       (The number of results to return, up to 50. Defaults to 10)

In [5]:
url = 'https://api.foursquare.com/v3/places/search'

params = {'ll': f'{df_NewYork_BikeStations.iloc[0,1]},{df_NewYork_BikeStations.iloc[0,2]}' ,
          'radius': 1000,
          'category': 'restaurant,coffee',  
          'max_price' : 3,
          'sort': 'RATING',
           'limit': 5
           }
response = requests.get(url, params=params, headers=headers)
data = response.json()
pprint(data)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 40.755226,
                                                  'longitude': -73.965826},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
                                       'suffix': '.png'},
                              'id': 13003,
                              'name': 'Bar',
                              'plural_name': 'Bars',
                              'short_name': 'Bar'},
                             {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_',
                                       'suffix': '.png'},
                              'id': 13276,
                              'name': 'Sushi Restaurant',
                              'plural_name': 'Sushi Restaurants',
                              'short_name': 'Sushi'},
                             {'icon': {'prefix': 'http

### 1.2.) Try the request with params containing ONLY: lat/long, radius, query
* New York bike station (lat/long)
* radius     : 1000m
* looking for: cinema,shopping

In [6]:
url = f'https://api.foursquare.com/v3/places/search?ll={df_NewYork_BikeStations.iloc[0,1]},{df_NewYork_BikeStations.iloc[0,2]}&radius=1000&query=cinema%20shopping'

response = requests.get(url, headers=headers)
data = response.json()
pprint(data)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 40.755226,
                                                  'longitude': -73.965826},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/realestate_',
                                       'suffix': '.png'},
                              'id': 11149,
                              'name': 'Real Estate Agency',
                              'plural_name': 'Real Estate Agencies',
                              'short_name': 'Real Estate'}],
              'chains': [],
              'closed_bucket': 'Unsure',
              'distance': 837,
              'fsq_id': '6e5c91376eee44931717c562',
              'geocodes': {'main': {'latitude': 40.759134,
                                    'longitude': -73.974337},
                           'roof': {'latitude': 40.759134,
                                    'longitude': -73.974337}}

### Comment:
#### ======> After Examining and comparing the results/structure between 1.1) and 1.2): it seems that for my location (New York city), this API has not yet covered the attributes like `rating, main/max_price` **even though its documentation says it does**. 

In [7]:
# list_data = data['results']
# pprint(list_data)

####

## 2) Generate Defined-function 
##### for Requesting Foursquare API to get information for **EACH** bike_station to get information
##### and Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc).

* My POIs here are: `restaurant,coffee,hotel,park,shopping,museum,cinema,bus,school`
* information I want to get: `name, address, distance` from the bike station

In [8]:
def FSQ_get_nearby_venues_for_each_bike_station(latitude, longitude, radius=1000, query=['restaurant', 'coffee','hotel','park','shopping','museum','cinema','bus','school']):
    # Requesting API to get information for **EACH** bike_station to get information
    url = 'https://api.foursquare.com/v3/places/search'
    params = {'ll': f'{latitude},{longitude}',
              'radius': radius,
              'query': ','.join(query),
              'sort': 'DISTANCE',
              }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    list_data = data['results']

    # Parse through the response to get needed information
    list_nearby_venues = []
    for place in list_data:
        
        # Handle some situation in which  no 'address' in JSON structure for POIs:
        add = "Empty"  if "address" not in place['location'].keys()  else  place['location']['address']     
        # Handle different situations of 'categories':
        cat = place['categories'][0]['name']  if len(place['categories']) >= 1  else  place['categories']
        
                                            # if len(place['categories']) >= 1: 
                                            #     cat = place['categories'][0]['name']
                                            # else: 
                                            #     cat = place['categories']
        # get needed information   
        venue_dict = {'venue_name': place['name'],
                      'venue_address': add,
                      'venue_distance': place['distance'],
                      'venue_category': cat
                      }
        list_nearby_venues.append(venue_dict)   # list contains many dicts; append into list
    return list_nearby_venues

In [9]:
# Test with  a bike station
list_nearby_venues_TEST  = FSQ_get_nearby_venues_for_each_bike_station(df_NewYork_BikeStations.iloc[0,1], df_NewYork_BikeStations.iloc[0,2])
list_nearby_venues_TEST 

[{'venue_name': 'The Pod Hotel 51',
  'venue_address': '230 E 51st St',
  'venue_distance': 278,
  'venue_category': 'Hotel'},
 {'venue_name': 'NYC Private Shopping Tour',
  'venue_address': '860 United Nations Plz',
  'venue_distance': 303,
  'venue_category': 'Tour Provider'},
 {'venue_name': 'Remi Flower & Coffee',
  'venue_address': '906 2nd Ave',
  'venue_distance': 318,
  'venue_category': 'Café'},
 {'venue_name': 'Birch Coffee',
  'venue_address': '240 E 54th St',
  'venue_distance': 320,
  'venue_category': 'Coffee Shop'},
 {'venue_name': 'Gregorys Coffee',
  'venue_address': '880 3rd Ave',
  'venue_distance': 442,
  'venue_category': 'Café'},
 {'venue_name': 'Weber Shandwick Coffee Bar',
  'venue_address': '909 3rd Ave',
  'venue_distance': 452,
  'venue_category': 'Coffee Shop'},
 {'venue_name': 'Ground Central Coffee Company',
  'venue_address': '155 E 52nd St',
  'venue_distance': 456,
  'venue_category': 'Wine Bar'},
 {'venue_name': 'San Carlos Hotel',
  'venue_address': '

## Implement def and Put the parsed results into a DataFrame

In [10]:
# Generate empty DataFrame
df_nearby_venues_Foursquare = pd.DataFrame(columns=['Name_Station', 'Venue_Name', 'Venue_Address', 'Venue_Distance', 'Venue_Category'])

In [11]:
for index, row in df_NewYork_BikeStations.iterrows():
    lat = row['Latitude_Station']
    long = row['Longitude_Station']
    
    list_nearby_venues_Foursquare  = FSQ_get_nearby_venues_for_each_bike_station(lat, long)
    for venue in list_nearby_venues_Foursquare:
        df_nearby_venues_Foursquare = pd.concat([df_nearby_venues_Foursquare,
                                                  pd.DataFrame({'Name_Station'  : [row['Name_Station']], 
                                                                'Venue_Name'    : [venue['venue_name']], 
                                                                'Venue_Address' : [venue['venue_address']],
                                                                'Venue_Distance' : [venue['venue_distance']],
                                                                'Venue_Category' : [venue['venue_category']]
                                                                })])
df_nearby_venues_Foursquare.reset_index(drop=True, inplace=True)
              
df_nearby_venues_Foursquare

,Name_Station,Venue_Name,Venue_Address,Venue_Distance,Venue_Category
0,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",The Pod Hotel 51,230 E 51st St,278,Hotel
1,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",NYC Private Shopping Tour,860 United Nations Plz,303,Tour Provider
2,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Remi Flower & Coffee,906 2nd Ave,318,Café
3,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Birch Coffee,240 E 54th St,320,Coffee Shop
4,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Gregorys Coffee,880 3rd Ave,442,Café
...,...,...,...,...,...
365,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",787 Coffee,159 E 10th St,452,Coffee Shop
366,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",Stuyvesant Park Residence - The New School,318 E 15th St,512,College Residence Hall
367,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",Cinema Studies Department,721 Broadway,572,College Academic Building
368,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",Blue Bottle Coffee,257 Park Ave S,596,"Cafe, Coffee, and Tea House"


### Store DataFrame into csv file

In [12]:
df_nearby_venues_Foursquare.to_csv('../data/Nearby_Venues_Foursquare.csv', index=False)

#

# B - YELP

## Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [13]:
# Call environment variable for YELP API key
access_token = os.environ["YELP_API_KEY"]

# Create dictionary for headers
headers = {"ccept": "application/json",
           "Authorization": "Bearer "+access_token}

url = 'https://api.yelp.com/v3/businesses/search'

## 1) EXPERIMENT with 1 bike_station to see the structure

### 1.1) Test with a LIST of POIs at one time

In [14]:
params = {'latitude': df_NewYork_BikeStations.iloc[0,1],
          'longitude' : df_NewYork_BikeStations.iloc[0,2],
          'radius': 1000,
          'term': 'hotel,park,restaurant,coffee,shopping,museum,cinema,bus,school',
          'sort_by': 'rating',
           }
response = requests.get(url, params=params, headers=headers)
data = response.json()
pprint(data)

{'businesses': [{'alias': 'grand-central-terminal-new-york',
                 'attributes': {'business_temp_closed': None,
                                'business_url': 'http://grandcentralterminal.com/',
                                'hot_and_new': None,
                                'waitlist_reservation': None},
                 'categories': [{'alias': 'landmarks',
                                 'title': 'Landmarks & Historical Buildings'},
                                {'alias': 'trainstations',
                                 'title': 'Train Stations'}],
                 'coordinates': {'latitude': 40.752727697036576,
                                 'longitude': -73.97723946032279},
                 'display_phone': '(212) 340-2583',
                 'distance': 1000.7128484227945,
                 'id': '8HkB3JfIj7_g-MN5ujoRig',
                 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/DzelM8YLNoywfnPIXxmHZA/o.jpg',
                 'is_closed': False,
 

### 1.2) Test with a SINGLE POI at one time

In [15]:
params = {'latitude': df_NewYork_BikeStations.iloc[0,1],
          'longitude' : df_NewYork_BikeStations.iloc[0,2],
          'radius': 1000,
          'term': 'restaurant',
          'price' : 3,    # means $$$
          'sort_by': 'rating',
           }
response = requests.get(url, params=params, headers=headers)
data = response.json()
pprint(data)

{'businesses': [{'alias': 'kjun-new-york',
                 'attributes': {'business_temp_closed': None,
                                'business_url': 'https://www.kjun.nyc',
                                'hot_and_new': None,
                                'menu_url': None,
                                'open24_hours': None,
                                'waitlist_reservation': None},
                 'categories': [{'alias': 'korean', 'title': 'Korean'},
                                {'alias': 'cajun', 'title': 'Cajun/Creole'}],
                 'coordinates': {'latitude': 40.74897, 'longitude': -73.97633},
                 'display_phone': '(347) 675-8026',
                 'distance': 1124.7126635861473,
                 'id': 'xQag1K9NezcXdXXn2AZcqA',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/7VA72QOSJaEQ4nMWXMPEqA/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '154 E 39th St',
                         

In [16]:
# list_data = data['businesses']
# pprint(list_data)

### Comment:
#### ======> After Examining and comparing TWO above results/structure, it seems that this API doesn't support to looking for many categories at a time.

####

## 2) Generate Defined-function 
####    for YELP API to get information for **EACH** bike_station to get information
####    and Parse through the response to get the POIs details you want (ratings, name, location, etc).

* For the analysis purpose, my POI will include: `'restaurant','coffee','bar','hotel','park','shopping','supermarket','museum','stadium','cinema', 'bus','school','hospital'`
* information I want to get: `name, address, distance` from the bike station, `phone, rating, price, review_count, url, category`.

In [17]:
def YELP_get_nearby_venues_for_each_bike_station(latitude, longitude, radius=1000, term='restaurant'):
    # Requesting API to get information for **EACH** bike_station to get information
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {'latitude': latitude,
              'longitude' : longitude,
              'radius': radius,
              'term': term,
              'price':[1,2,3,4],
              'sort_by': 'rating'
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    list_data = data['businesses']

    # Parse through the response to get needed information
    list_nearby_venues = []
    for place in list_data:
        place_dict = {'venue_name': place['name'],
                      'venue_address': place['location']['address1'],
                      'venue_distance': place['distance'],
                      'venue_phone' : place['display_phone'],
                      'venue_rating' : place['rating'],
                      'venue_price' : place['price'],
                      'venue_review_count' : place['review_count'],
                      'venue_url' : place['attributes']['business_url'],
                      'venue_subcategory' : place['categories'][0]['title']
                      }
        list_nearby_venues.append(place_dict)   # list contains appends many dicts
    return list_nearby_venues

In [18]:
# Test with a bike station
list_nearby_venues_TEST  = YELP_get_nearby_venues_for_each_bike_station(df_NewYork_BikeStations.iloc[0,1], df_NewYork_BikeStations.iloc[0,2])
list_nearby_venues_TEST    # print to see surely

[{'venue_name': 'Village Crown Kosher Catering',
  'venue_address': '216 E 49th St',
  'venue_distance': 401.3791707065456,
  'venue_phone': '(212) 207-3888',
  'venue_rating': 4.9,
  'venue_price': '$$',
  'venue_review_count': 9,
  'venue_url': 'http://www.villagecrown.com',
  'venue_subcategory': 'Caterers'},
 {'venue_name': 'Charoen Krung',
  'venue_address': '955 2nd Ave',
  'venue_distance': 227.25238688478822,
  'venue_phone': '(929) 789-9456',
  'venue_rating': 4.8,
  'venue_price': '$$',
  'venue_review_count': 121,
  'venue_url': 'http://www.charoenkrungnyc.com',
  'venue_subcategory': 'Thai'},
 {'venue_name': 'Quick Stop',
  'venue_address': '47th Street & Lexington Ave',
  'venue_distance': 672.8227154763755,
  'venue_phone': '(917) 577-7057',
  'venue_rating': 4.8,
  'venue_price': '$',
  'venue_review_count': 9,
  'venue_url': 'http://georzes.wix.com/quickshop1',
  'venue_subcategory': 'Coffee & Tea'},
 {'venue_name': 'Kati Shop',
  'venue_address': '162 E 55th St',
  've

## Implement def and Put the parsed results into a DataFrame

In [19]:
# Generate empty DataFrame
df_nearby_venues_YELP = pd.DataFrame(columns=['Name_Station', 'Venue_Name', 'Venue_Address', 'Venue_Distance', 'Venue_Phone',
                                              'Venue_Rating', 'Venue_Price', 'Venue_Review_Count', 'Venue_URL', 'Venue_Subcategory', 'Venue_Category'])
# Generate a list containing POIs looking for
list_terms = ['restaurant','coffee', 'bar', 'hotel','park', 'shopping','supermarket','museum','stadium','cinema','bus','school','hospital']

In [20]:
#  Iterate through each bike station
for index, row in df_NewYork_BikeStations.iterrows():
    lat = row['Latitude_Station']
    long = row['Longitude_Station']

    # Iterate through each term searching for
    for term in list_terms:
        list_nearby_venues_YELP = YELP_get_nearby_venues_for_each_bike_station(lat, long, term=term)

        # Iterate through each venue to get information
        for venue in list_nearby_venues_YELP:
            df_nearby_venues_YELP = pd.concat([df_nearby_venues_YELP,
                                                      pd.DataFrame({'Name_Station'  : [row['Name_Station']], 
                                                                    'Venue_Name'    : [venue['venue_name']], 
                                                                    'Venue_Address' : [venue['venue_address']],
                                                                    'Venue_Distance': [venue['venue_distance']],
                                                                    'Venue_Phone'   : [venue['venue_phone']],
                                                                    'Venue_Rating'  : [venue['venue_rating']],
                                                                    'Venue_Price'   : [venue['venue_price']],
                                                                    'Venue_Review_Count' : [venue['venue_review_count']],
                                                                    'Venue_URL'     : [venue['venue_url']],
                                                                    'Venue_Subcategory' : [venue['venue_subcategory']],
                                                                    'Venue_Category' : [term]
                                                                    })])
            df_nearby_venues_YELP.reset_index(drop=True, inplace=True)
       
              
df_nearby_venues_YELP

,Name_Station,Venue_Name,Venue_Address,Venue_Distance,Venue_Phone,Venue_Rating,Venue_Price,Venue_Review_Count,Venue_URL,Venue_Subcategory,Venue_Category
0,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Village Crown Kosher Catering,216 E 49th St,401.379171,(212) 207-3888,4.9,$$,9,http://www.villagecrown.com,Caterers,restaurant
1,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Charoen Krung,955 2nd Ave,227.252387,(929) 789-9456,4.8,$$,121,http://www.charoenkrungnyc.com,Thai,restaurant
2,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Quick Stop,47th Street & Lexington Ave,672.822715,(917) 577-7057,4.8,$,9,http://georzes.wix.com/quickshop1,Coffee & Tea,restaurant
3,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Kati Shop,162 E 55th St,495.560320,(646) 869-1362,4.7,$$,212,http://www.katishopnyc.com,Thai,restaurant
4,"Icon Pkg: E 52 St, 1st Ave & 2nd Ave",Tony's Coffee Cart,"Sw Corner, 49th St & Park Ave",732.206967,,4.7,$,7,None,Coffee & Tea,restaurant
...,...,...,...,...,...,...,...,...,...,...,...
5254,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",PhotoManhattan,51 W 14th St,648.640472,(212) 929-3302,4.2,$$,61,http://www.photomanhattan.com/,Specialty Schools,school
5255,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",Make Up For Ever,8 E 12th St,372.101711,(212) 941-9337,4.2,$$$,60,https://www.makeupforever.com/us/en,Cosmetics & Beauty Supply,school
5256,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave",The Yinova Center,37 W 17th St,712.220500,(212) 533-2255,4.6,$$,51,https://www.yinovacenter.com,Acupuncture,hospital
5257,"Icon Pkg: E 13 St, 4th Ave & 3rd Ave","Leeber Cohen, MD",11 5th Ave,530.179160,(212) 777-1644,4.1,$,12,None,Medical Spas,hospital


### Store DataFrame into csv file

In [21]:
df_nearby_venues_YELP.to_csv('../data/Nearby_Venues_YELP.csv', index=False)

In [22]:
df_nearby_venues_YELP.shape

(5259, 11)

#

# C- Comparing Results

#### 1. Which API provided you with more complete data? Provide an explanation. 

For my location (New York city), **YELP API** provided me with more complete data than FOURSQUARE API.
 - FOURSQUARE API: has not yet covered the attributes I want, such as rating, main/max_price,etc... even though its documentation says it does.
 - YELP API: provides a lot of different interesting attributes for each POI, such as review_count, rating, price, phone, even business url, menu url, etc...
     - However, YELP API has one drawback. It doesn't allow to query 2 terms at one time like FOURSQUARE API, for example:
       -  With FOURSQUARE API, we can search 'restaurant,coffees' at one time;
        - while with YELP API, we can only search 'restaurant' then later search 'coffee' , but I am still fine with that.

#### 2. Get the top 10 restaurants according to their rating

In [23]:
df_nearby_venues_YELP[df_nearby_venues_YELP['Venue_Category']=='restaurant'].sort_values(by='Venue_Rating', ascending=False).head(10)


,Name_Station,Venue_Name,Venue_Address,Venue_Distance,Venue_Phone,Venue_Rating,Venue_Price,Venue_Review_Count,Venue_URL,Venue_Subcategory,Venue_Category
428,Icon Pkg: King Street & Greenwich St,El Jalapeno taqueria,204 W 4th St,731.556787,(347) 697-0081,5.0,$,14,None,Mexican,restaurant
2236,"Laz Pkg: W 23 St, 6th & 7th Ave",Loopy's Eatery,156 W 29th St,474.759283,(212) 381-0060,5.0,$$,9,https://www.loopyseatery.com,Korean,restaurant
4961,JOCO X Grubhub Concierge,Golden Eggies,39 Ave A,703.566429,(646) 387-3684,5.0,$,25,None,Waffles,restaurant
3902,"Icon Pkg: W 14 St, 8th Ave & 9th Ave",El Jalapeno taqueria,204 W 4th St,771.190175,(347) 697-0081,5.0,$,14,None,Mexican,restaurant
3903,"Icon Pkg: W 14 St, 8th Ave & 9th Ave",Mr. Jones Supper Club,22 Jones St,928.322987,,5.0,$$$$,70,https://www.misterjonesnyc.com,Australian,restaurant
4962,JOCO X Grubhub Concierge,Halal Food,95 East Houston St,1048.195224,,5.0,$,5,None,Halal,restaurant
140,"Icon Pkg: W 29 St, 11th Ave & 10th Ave",Loopy's Eatery,156 W 29th St,1121.528814,(212) 381-0060,5.0,$$,9,https://www.loopyseatery.com,Korean,restaurant
139,"Icon Pkg: W 29 St, 11th Ave & 10th Ave",Kaptan's Deck,337 W 30th St,640.838862,,5.0,$$$$,5,,Bars,restaurant
1052,"Icon Pkg: W 37 St, 9th & 10th Ave",Loopy's Eatery,156 W 29th St,936.586697,(212) 381-0060,5.0,$$,9,https://www.loopyseatery.com,Korean,restaurant
429,Icon Pkg: King Street & Greenwich St,Mr. Jones Supper Club,22 Jones St,624.945637,,5.0,$$$$,70,https://www.misterjonesnyc.com,Australian,restaurant


### Note: As we can see the result above, there are really duplicated restaurants 
 - because the 'Name_Station"s are different (means that multiple bike stations can have the same venue).
 - so, below result is not what we expected, we should do a little more as follows:

In [24]:
# Delete 2 columns and then delete duplicated rows
df = df_nearby_venues_YELP.drop(['Name_Station','Venue_Distance'], axis=1)
df.shape

(5259, 9)

In [25]:
df.drop_duplicates(keep='first', inplace=True)
df.shape

(1838, 9)

In [26]:
top10_rating_restaurants = df[df['Venue_Category']=='restaurant'].sort_values(by='Venue_Rating', ascending=False).head(10)
top10_rating_restaurants.reset_index(drop=True, inplace=True)

print("Top 10 rating restaurants are:\n")
top10_rating_restaurants

Top 10 rating restaurants are:



,Venue_Name,Venue_Address,Venue_Phone,Venue_Rating,Venue_Price,Venue_Review_Count,Venue_URL,Venue_Subcategory,Venue_Category
0,El Jalapeno taqueria,204 W 4th St,(347) 697-0081,5.0,$,14,None,Mexican,restaurant
1,Halal Food,95 East Houston St,,5.0,$,5,None,Halal,restaurant
2,Golden Eggies,39 Ave A,(646) 387-3684,5.0,$,25,None,Waffles,restaurant
3,Mr. Jones Supper Club,22 Jones St,,5.0,$$$$,70,https://www.misterjonesnyc.com,Australian,restaurant
4,Basmala Halal Food,NE Corner of Broadway & W 96th St,,5.0,$,4,,Food Trucks,restaurant
5,Loopy's Eatery,156 W 29th St,(212) 381-0060,5.0,$$,9,https://www.loopyseatery.com,Korean,restaurant
6,Kaptan's Deck,337 W 30th St,,5.0,$$$$,5,,Bars,restaurant
7,Village Crown Kosher Catering,216 E 49th St,(212) 207-3888,4.9,$$,9,http://www.villagecrown.com,Caterers,restaurant
8,Farook Halal Food,142 E 14th St,(917) 943-2583,4.9,$,63,,Halal,restaurant
9,ME Chicken Rice,120 West 3rd St,(646) 455-0877,4.9,$$,128,https://mechickenrice.square.site,Thai,restaurant
